## Problem statement

Next, take one of the samples from training dataset and use PINN [3] for estimating the value of
G. Report the CPU/GPU time taken. You may use equation 4 for computing the residual loss.

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

### Device and seed

In [43]:
# device = tf.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Device used: {device}")
# np.random.seed(seed=1234)
seed=5
tf.random.set_seed(seed)
tf.config.list_physical_devices('GPU')
picked = tf.random.uniform([1],0,400,dtype='int32')

### Load Data

In [89]:
## Load dataset
relative_csv_path = "./../src/data/raw/Dataset.csv"
split_ratio = 0.2

# Load different temperatures samples 
fields = ['T1', 'T2', 'T3', 'T4' ,'T5', 'T6' ,'T7', 'T8' ,'T9']
df_T = pd.read_csv(relative_csv_path, skipinitialspace=True, usecols=fields)
T_train=df_T.to_numpy()[picked,]

T_inf = 27 + 273 #in K
T_b = 127 + 273 #in K
Theta = np.multiply(np.add(T, [-T_inf]), [1/(T_b - T_inf)])
Theta = Theta.T

# Load different G values 
fields = ['G']
df_G = pd.read_csv(relative_csv_path, skipinitialspace=True, usecols=fields)
G_test = df_G.to_numpy()[picked,]

## $X^*_T$ and  $X^*_F$

In [113]:
Nu = Theta.shape[0]
Nf = 50
X_T = np.linspace(0,1,Nu).reshape(-1,1)
X_F = np.linspace(0,1,Nf).reshape(-1,1)

## Transforming into TensorFlow

In [119]:
Theta_tf = tf.convert_to_tensor(Theta, dtype=tf.float32)
X_T_tf = tf.convert_to_tensor(X_T, dtype=tf.float32)
X_F_tf = tf.convert_to_tensor(X_F, dtype=tf.float32)
x0 = X_T_tf[0] # X* = 0 para la boundary condition
x1 = X_T_tf[-1] # X* = 1 para la boundary condition
Theta_tf[0]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [117]:
tf.config.experimental.enable_tensor_float_32_execution(False)
#os.environ[‘TF_ENABLE_AUTO_MIXED_PRECISION’] = ‘1’

# Initalization of Network
def hyper_initial(size):
    """
    Initilizes the layer weights according to Xavier procedure. 
    
    Parameters:
       size (integer): Input size.
    
    """
    in_dim = size[0]
    out_dim = size[1]
    std = np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.random.truncated_normal(shape=size, stddev = std))

# Neural Network 
def DNN(X, W, b):
    A = X
    L = len(W)
    for i in range(L-1):
        A = tf.tanh(tf.add(tf.matmul(A, W[i]), b[i]))
    Y = tf.add(tf.matmul(A, W[-1]), b[-1])
    return Y

def train_vars(W, b):
    return W + b

# def net_u(x, t, w, b):
#     u = DNN(tf.concat([x,t],1), w, b)
#     return u

def net_u(x, w, b):
    out_net_u = DNN(x, w, b)
    N, G, eg, ec, theta = out_net_u[0], out_net_u[1], out_net_u[2], out_net_u[3], out_net_u[4] # probar sacar derechito
    return N, G, eg, ec, theta


#@tf.function(jit_compile=True)
# @tf.function
# def net_f(x,t,W, b, nu):
#     with tf.GradientTape(persistent=True) as tape1:
#         tape1.watch([x, t])
#         with tf.GradientTape(persistent=True) as tape2:
#             tape2.watch([x, t])
#             u=net_u(x,t, W, b)
#         u_t = tape2.gradient(u, t)
#         u_x = tape2.gradient(u, x)
#     u_xx = tape1.gradient(u_x, x)  
#     del tape1
#     f = u_t + u*u_x - nu*u_xx
#     return f

@tf.function
def net_f(x, W, b):
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch(x)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            N, G, eg, ec, theta = net_u(x, W, b)
            theta_x = tape2.gradient(theta, x)
            aux = (1 + ec * theta) * theta_x      
    aux_x = tape1.gradient(aux, x)  
    del tape1
    f = aux_x - N**2*theta + N**2*G*(1 + eg*theta) 
    return f


@tf.function
def net_bc(x, W, b):
    with tf.GradientTape(persistent=True) as tape3:
        tape3.watch(x)
        _, _, _, _, theta = net_u(x, W, b)
        theta_x = tape3.gradient(theta, x)
    del tape3
    f = theta_x 
    return f


#@tf.function(jit_compile=True)
@tf.function

Theta_tf = tf.convert_to_tensor(Theta, dtype=tf.float32)
X_T_tf = tf.convert_to_tensor(X_T, dtype=tf.float32)
X_F_tf = tf.convert_to_tensor(X_F, dtype=tf.float32)
x0=X_T_tf[0] # X* = 0 para la boundary condition
x1=X_T_tf[-1] # X* = 1 para la boundary condition


def train_step(W, b, X_T_tf, Theta_tf, X_F_tf, x1, opt):
    x_u = X_T_tf[:,0:1] 
    x_f = X_F_tf[:,0:1]
    with tf.GradientTape() as tape:
        tape.watch([W,b])
        N_nn, G_nn, eg_nn, ec_nn, theta_nn = net_u(X_T_tf, W, b) 
        f_nn = net_f(X_F_tf, W, b)
        bc_nn = net_bc(x1, W, b)
        loss_T = tf.reduce_mean(tf.square(theta_nn - Theta_tf)) 
        loss_f = tf.reduce_mean(tf.square(f_nn))
        loss_bc = tf.reduce_mean(tf.square(bc_nn))
        loss =  loss_T + loss_f + loss_bc  
    grads = tape.gradient(loss, train_vars(W,b))
    opt.apply_gradients(zip(grads, train_vars(W,b)))
    return loss


    
# nu = 0.01/np.pi
# noise = 0.0        
# N_u = 100
# N_f = 10000
# Nmax=40000

# layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
# L = len(layers)
# W = [hyper_initial([layers[l-1], layers[l]]) for l in range(1, L)] 
# b = [tf.Variable(tf.zeros([1, layers[l]])) for l in range(1, L)] 

# data = scipy.io.loadmat('./Data/burgers_shock.mat')

# t = data['t'].flatten()[:,None]
# x = data['x'].flatten()[:,None]
# Exact = np.real(data['usol']).T
# X, T = np.meshgrid(x,t)
# X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
# u_star = Exact.flatten()[:,None]              
# # Doman bounds
# lb = X_star.min(0)
# ub = X_star.max(0)    
# xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
# uu1 = Exact[0:1,:].T
# xx2 = np.hstack((X[:,0:1], T[:,0:1]))
# uu2 = Exact[:,0:1]
# xx3 = np.hstack((X[:,-1:], T[:,-1:]))
# uu3 = Exact[:,-1:]

# X_u_train = np.vstack([xx1, xx2, xx3])
# X_f_train = lb + (ub-lb)*lhs(2, N_f)
# X_f_train = np.vstack((X_f_train, X_u_train))
# u_train = np.vstack([uu1, uu2, uu3])

# idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)

# X_u_train = X_u_train[idx, :]
# u_train = u_train[idx,:]

# X_u_train_tf = tf.convert_to_tensor(X_u_train, dtype=tf.float32)
# u_train_tf =   tf.convert_to_tensor(u_train, dtype=tf.float32)
# X_f_train_tf = tf.convert_to_tensor(X_f_train, dtype=tf.float32)

# lr = 1e-3
# optimizer = tf.optimizers.Adam(learning_rate=lr)

# start_time = time.time()
# n=0
# loss = []
# while n <= Nmax:
#     loss_= train_step(W, b, X_u_train_tf, u_train_tf, X_f_train_tf, optimizer, nu)
#     loss.append(loss_)    
#     print(f"Iteration is: {n} and loss is: {loss_}")
#     n+=1

# elapsed = time.time() - start_time                
# print('Training time: %.4f' % (elapsed))


NameError: name 'scipy' is not defined